In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/weather-prediction/weather_prediction_dataset.csv
/kaggle/input/weather-prediction/metadata.txt
/kaggle/input/weather-prediction/weather_prediction_bbq_labels.csv


In this notebook, we use Spark Machine Learning library to work on this dataset. A related notebook that explored this dataset using ML techniques from sklearn and ANN/CNN from tensorflow can be found here:

https://www.kaggle.com/code/lorresprz/ann-cnn-randomforest-predicting-nice-weather



In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=39a4cb35dec5b1cf8fef2a22afe46472512e90d5b0a7e6c98a6df634b00b7317
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('weather_pred').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 07:13:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df1 = spark.read.csv('/kaggle/input/weather-prediction/weather_prediction_dataset.csv', header=True, inferSchema=True)
df2 = spark.read.csv('/kaggle/input/weather-prediction/weather_prediction_bbq_labels.csv',header=True, inferSchema=True)

There are 165 columns for the 18 cities in Europe. For each cities, multiple measurements including wind gust, wind speed, cloud cover, humidity, pressure, global radiation, precipitation, sunshine, minimum temperature, maximum temperature were recorded. Furthermore, the set of measurements varies from city to city (for example, some city might have wind speed in place of cloud cover or some might have wind gust in place of wind speed). For this notebook, we will focus on Maastricht, a beautiful Dutch city. The objective of this is to predict whether the weather is okay for to have a barbecue outdoors.

In [7]:
df1.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- BASEL_cloud_cover: integer (nullable = true)
 |-- BASEL_humidity: double (nullable = true)
 |-- BASEL_pressure: double (nullable = true)
 |-- BASEL_global_radiation: double (nullable = true)
 |-- BASEL_precipitation: double (nullable = true)
 |-- BASEL_sunshine: double (nullable = true)
 |-- BASEL_temp_mean: double (nullable = true)
 |-- BASEL_temp_min: double (nullable = true)
 |-- BASEL_temp_max: double (nullable = true)
 |-- BUDAPEST_cloud_cover: integer (nullable = true)
 |-- BUDAPEST_humidity: double (nullable = true)
 |-- BUDAPEST_pressure: double (nullable = true)
 |-- BUDAPEST_global_radiation: double (nullable = true)
 |-- BUDAPEST_precipitation: double (nullable = true)
 |-- BUDAPEST_sunshine: double (nullable = true)
 |-- BUDAPEST_temp_mean: double (nullable = true)
 |-- BUDAPEST_temp_max: double (nullable = true)
 |-- DE_BILT_cloud_cover: integer (nullable = true)
 |-- DE_BILT_wind_speed: d

In [8]:
df2.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- BASEL_BBQ_weather: boolean (nullable = true)
 |-- BUDAPEST_BBQ_weather: boolean (nullable = true)
 |-- DE_BBQ_weather: boolean (nullable = true)
 |-- DRESDEN_BBQ_weather: boolean (nullable = true)
 |-- DUSSELDORF_BBQ_weather: boolean (nullable = true)
 |-- HEATHROW_BBQ_weather: boolean (nullable = true)
 |-- KASSEL_BBQ_weather: boolean (nullable = true)
 |-- LJUBLJANA_BBQ_weather: boolean (nullable = true)
 |-- MAASTRICHT_BBQ_weather: boolean (nullable = true)
 |-- MALMO_BBQ_weather: boolean (nullable = true)
 |-- MONTELIMAR_BBQ_weather: boolean (nullable = true)
 |-- MUENCHEN_BBQ_weather: boolean (nullable = true)
 |-- OSLO_BBQ_weather: boolean (nullable = true)
 |-- PERPIGNAN_BBQ_weather: boolean (nullable = true)
 |-- SONNBLICK_BBQ_weather: boolean (nullable = true)
 |-- STOCKHOLM_BBQ_weather: boolean (nullable = true)
 |-- TOURS_BBQ_weather: boolean (nullable = true)



In [9]:
#pick the columns corresponding to Maastricht
df1.columns[80:91]

['MAASTRICHT_cloud_cover',
 'MAASTRICHT_wind_speed',
 'MAASTRICHT_wind_gust',
 'MAASTRICHT_humidity',
 'MAASTRICHT_pressure',
 'MAASTRICHT_global_radiation',
 'MAASTRICHT_precipitation',
 'MAASTRICHT_sunshine',
 'MAASTRICHT_temp_mean',
 'MAASTRICHT_temp_min',
 'MAASTRICHT_temp_max']

In [10]:
dfM = df1.select(['DATE','MAASTRICHT_cloud_cover',
 'MAASTRICHT_wind_speed',
 'MAASTRICHT_wind_gust',
 'MAASTRICHT_humidity',
 'MAASTRICHT_pressure',
 'MAASTRICHT_global_radiation',
 'MAASTRICHT_precipitation',
 'MAASTRICHT_sunshine',
 'MAASTRICHT_temp_mean',
 'MAASTRICHT_temp_min',
 'MAASTRICHT_temp_max'])

In [11]:
dfM2 = df2.select(['DATE','MAASTRICHT_BBQ_weather'])
dfM2.show()

+--------+----------------------+
|    DATE|MAASTRICHT_BBQ_weather|
+--------+----------------------+
|20000101|                 false|
|20000102|                 false|
|20000103|                 false|
|20000104|                 false|
|20000105|                 false|
|20000106|                 false|
|20000107|                 false|
|20000108|                 false|
|20000109|                 false|
|20000110|                 false|
|20000111|                 false|
|20000112|                 false|
|20000113|                 false|
|20000114|                 false|
|20000115|                 false|
|20000116|                 false|
|20000117|                 false|
|20000118|                 false|
|20000119|                 false|
|20000120|                 false|
+--------+----------------------+
only showing top 20 rows



In [12]:
#Join the labels (True, False) from the second newly created dataframe to the first one
dfM2= dfM2.join(dfM, on=["DATE"])
dfM2.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- MAASTRICHT_BBQ_weather: boolean (nullable = true)
 |-- MAASTRICHT_cloud_cover: integer (nullable = true)
 |-- MAASTRICHT_wind_speed: double (nullable = true)
 |-- MAASTRICHT_wind_gust: double (nullable = true)
 |-- MAASTRICHT_humidity: double (nullable = true)
 |-- MAASTRICHT_pressure: double (nullable = true)
 |-- MAASTRICHT_global_radiation: double (nullable = true)
 |-- MAASTRICHT_precipitation: double (nullable = true)
 |-- MAASTRICHT_sunshine: double (nullable = true)
 |-- MAASTRICHT_temp_mean: double (nullable = true)
 |-- MAASTRICHT_temp_min: double (nullable = true)
 |-- MAASTRICHT_temp_max: double (nullable = true)



In [13]:
#The data format required by spark ML is (features, label) so we need to assemble all features into a feature vector. 
from pyspark.ml.feature import VectorAssembler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
#create an instance of the Assembler which takes in a series of columns to be used as features
#and returns a condensed vector
assembler = VectorAssembler(inputCols = ['DATE','MAASTRICHT_cloud_cover',
 'MAASTRICHT_wind_speed',
 'MAASTRICHT_wind_gust',
 'MAASTRICHT_humidity',
 'MAASTRICHT_pressure',
 'MAASTRICHT_global_radiation',
 'MAASTRICHT_precipitation',
 'MAASTRICHT_sunshine',
 'MAASTRICHT_temp_mean',
 'MAASTRICHT_temp_min',
 'MAASTRICHT_temp_max',
 'MAASTRICHT_cloud_cover',
 'MAASTRICHT_wind_speed',
 'MAASTRICHT_wind_gust',
 'MAASTRICHT_humidity',
 'MAASTRICHT_pressure',
 'MAASTRICHT_global_radiation',
 'MAASTRICHT_precipitation',
 'MAASTRICHT_sunshine',
 'MAASTRICHT_temp_mean',
 'MAASTRICHT_temp_min',
 'MAASTRICHT_temp_max'], outputCol = 'features')

In [15]:
#Call the 'transform' method on the dataframe returns a new dataframe with the newly created 'features' column

data = assembler.transform(dfM2)

In [21]:
#transform the labels 'bbq_weather' to numerics
from pyspark.sql.functions import col
dfM2.select([col(c).cast("integer") for c in ["MAASTRICHT_BBQ_weather"]])

DataFrame[MAASTRICHT_BBQ_weather: int]

In [26]:
final_data = data.select(['features', [col(c).cast("integer") for c in ["MAASTRICHT_BBQ_weather"]]])
final_data.show()

PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got list.

In [23]:
final_datfinal_data.select([])

DataFrame[MAASTRICHT_BBQ_weather: int]

In [24]:
final_data.show()

+--------------------+----------------------+
|            features|MAASTRICHT_BBQ_weather|
+--------------------+----------------------+
|[2.0000101E7,8.0,...|                 false|
|[2.0000102E7,7.0,...|                 false|
|[2.0000103E7,7.0,...|                 false|
|[2.0000104E7,8.0,...|                 false|
|[2.0000105E7,4.0,...|                 false|
|[2.0000106E7,6.0,...|                 false|
|[2.0000107E7,6.0,...|                 false|
|[2.0000108E7,7.0,...|                 false|
|[2.0000109E7,6.0,...|                 false|
|[2.000011E7,7.0,1...|                 false|
|[2.0000111E7,3.0,...|                 false|
|[2.0000112E7,5.0,...|                 false|
|[2.0000113E7,8.0,...|                 false|
|[2.0000114E7,8.0,...|                 false|
|[2.0000115E7,8.0,...|                 false|
|[2.0000116E7,8.0,...|                 false|
|[2.0000117E7,8.0,...|                 false|
|[2.0000118E7,8.0,...|                 false|
|[2.0000119E7,7.0,...|            